In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchsummary import summary
from torcheval.metrics import BinaryPrecisionRecallCurve
import torchmetrics
from tqdm.notebook import tqdm
from torchmetrics import R2Score
from sklearn.metrics import precision_recall_curve, f1_score, classification_report
from comet_ml import Experiment
from sklearn import preprocessing 

%matplotlib inline

In [ ]:
N=10
df=pd.read_csv('datasets/data.csv')
df=np.log(df[['e_mu_current', 'e_nu_1', 'e_nu_2', 'q_mu', 'q_nu1', 'q_nu2']])
df=df.rolling(window=N).mean()
df=df.iloc[N:]
df[['e_mu_current', 'e_nu_1', 'e_nu_2', 'q_mu', 'q_nu1', 'q_nu2']]=preprocessing.MinMaxScaler((0,1)).fit_transform(df[['e_mu_current', 'e_nu_1', 'e_nu_2', 'q_mu', 'q_nu1', 'q_nu2']])
df

In [ ]:
plt.plot(df['e_nu_2'])
plt.show()

In [ ]:
df[['e_nu_1', 'e_nu_2', 'q_mu', 'q_nu1', 'q_nu2']].values

In [ ]:
df[['e_nu_1', 'e_nu_2', 'q_mu', 'q_nu1', 'q_nu2']].values.reshape((-1,1,10))[0]

In [ ]:
class BoxDataset(Dataset):
    
    @classmethod
    def __init__(self, df_path,N=10, windth=1):
        df=pd.read_csv(df_path)
        df=np.log(df)
        df=df.rolling(window=N).mean()
        df=df.iloc[N:]
        df[['e_mu_current', 'e_nu_1', 'e_nu_2', 'q_mu', 'q_nu1', 'q_nu2']]=preprocessing.MinMaxScaler((0,1)).fit_transform(df[['e_mu_current', 'e_nu_1', 'e_nu_2', 'q_mu', 'q_nu1', 'q_nu2']])
        self.data=df[['e_nu_1', 'e_nu_2', 'q_mu', 'q_nu1', 'q_nu2']].values[:-1]
        
        self.vectors = []
        for i, point in enumerate(self.data[:-windth]):
            self.vectors.append(self.data[i:windth+i,:].flatten())
        self.vectors=np.array(self.vectors)
        self.target=df['e_mu_current'].values[windth:]
        
        self.vectors = torch.FloatTensor(self.vectors)
        self.target = torch.FloatTensor(self.target)
    def __len__(self):
        return len(self.vectors)

    def __getitem__(self, idx):
        return {'vectors':self.vectors[idx], 'target':self.target[idx]}

In [ ]:
width=10
dataset=BoxDataset(df_path='datasets/data.csv',windth=width)

val_sptit = 0.2
batch_size = 512

train_len = len(dataset) - int(len(dataset) * val_sptit)
test_len = int(len(dataset) * val_sptit)

train_set, val_set = torch.utils.data.random_split(dataset, [train_len, test_len])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True,
                          drop_last=True)
test_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False,
                         drop_last=True)

In [ ]:
class Net(nn.Module):
    def __init__(self,in_channels=1):
        super(Net, self).__init__()
        
        self.relu = nn.LeakyReLU()

        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=16,kernel_size=3)
        self.batch_norm1 = nn.BatchNorm1d(16)
        self.maxpool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32,kernel_size=3)
        self.batch_norm2 = nn.BatchNorm1d(32)
        self.maxpool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64,kernel_size=3)
        self.batch_norm3 = nn.BatchNorm1d(64)
        self.maxpool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128,kernel_size=3)
        self.batch_norm4 = nn.BatchNorm1d(128)
        self.maxpool4 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.fc1_1 = nn.Linear(in_features=1472, out_features=256)
        self.anomaly = nn.Linear(in_features=256, out_features=1)


        for tensor in [self.conv1, self.conv2, self.conv3, self.conv4, self.fc1_1, self.anomaly]:
            torch.nn.init.kaiming_uniform_(tensor.weight)

    def forward(self, x):

        x = self.conv1(x)
        x = self.relu(x)
        x = self.batch_norm1(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.batch_norm2(x)

        x = torch.flatten(x, 1)
        output1_1 = self.fc1_1(x)
        anomaly = self.anomaly(output1_1)
        anomaly=nn.Sigmoid()(anomaly)

        return anomaly

In [ ]:
model = Net()

res = model.cuda()
summary(model, (1,int(5*width)))

In [ ]:
# log=False
log=True
if log==True:
    experiment = Experiment(
        api_key="5skWtD5KzMcU6rVhVNOLfQZ6E",
        project_name="qber-nn",
        workspace="dkagramanyan",
    )

device = 'cuda'
# device = 'cpu'

model.train()
model.to(device)

learning_rate = 0.00001
# learning_rate = 0.1 * (batch_size / 256)

mse_loss=nn.MSELoss()

acc_metric=torchmetrics.classification.MulticlassAccuracy(num_classes=5)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

epochs_cnt = 10000
tbar = tqdm(range(epochs_cnt), leave=True, desc='?')


for epoch in tbar:
    for i, frames_series in enumerate(train_loader):
        vectors = frames_series['vectors'].to(device).reshape((-1,1,int(5*width)))
        target = frames_series['target'].to(device)

        target_output = model(vectors)

        # MSe change point classification
        loss = mse_loss(torch.squeeze(target_output), target).to(device)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if epoch % 5 == 0:
        with torch.no_grad():
            r2score = R2Score().to(device)
            l1=nn.L1Loss()
            
            loss_list=[]
            r2score_list=[]
            l1_loss_list=[]

            for j, test_frames_series in enumerate(test_loader):
                vectors = test_frames_series['vectors'].to(device).reshape((-1,1,int(5*width)))
                target = test_frames_series['target'].to(device)
        
                target_output = model(vectors)
        
                # BCE change point classification
                loss = mse_loss(torch.squeeze(target_output), target).to(device)
                loss_list.append(loss)
                
                # R2 metrics
                r2score_metrics=r2score(torch.squeeze(target_output), target)
                r2score_list.append(r2score_metrics)
                
                # L1 metrics
                l1_loss=l1(torch.squeeze(target_output), target)
                l1_loss_list.append(l1_loss)

            loss_t=torch.mean(torch.FloatTensor(loss_list))
            r2score_t=torch.mean(torch.FloatTensor(r2score_list))
            l1_t=torch.mean(torch.FloatTensor(l1_loss_list))
            
            msg = f'\nTest MSE Loss: {loss_t:.4f}\tR2 metrics: {r2score_t:.4f}\tL1 loss: {l1_t:.6f}'

            print(msg)
            if log==True:
                experiment.log_metric('Train MSE loss', loss.item())
                experiment.log_metric('Test MSE loss', loss_t)
                experiment.log_metric('Test R2  metrics', r2score_t)
                experiment.log_metric('Test L1 loss', l1_t)



In [ ]:
experiment.end()

In [ ]:
torch.save(model,'width3.pt')

In [ ]:
predicted=[]
true=[]
model.cpu()

with torch.no_grad():
    for data in tqdm(dataset):
        true.append(data['target'])
        
        pred=model(data['vectors'].reshape((1,1,15)))
        predicted.append(pred[0][0])
        

In [ ]:
start=32000
end=34000

plt.figure(figsize=(15,5))
plt.plot(true[start:end])
plt.plot(predicted[start:end])
plt.legend(['True', 'Predicted'])
plt.show()